# LIBRARY BLOCK

In [1]:
from selenium import webdriver
import json
import time
from time import sleep
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import pandas as pd

# SET UP BLOCK

In [2]:
NUMBER_OF_YEAR = 8

# FUNCTION BLOCK

In [3]:
def year_choosing():
    year_button = driver.find_element(By.CSS_SELECTOR, 'input[onchange="redirectFromSelect2(this.value,0)"]')
    year_button.click()
    sleep(2)

In [4]:
def year_monitoring(move_type, times=1):
    if move_type == "before":
        for _ in range(times):
            year_before = driver.find_element(By.CSS_SELECTOR, 'img[alt="Xem dữ liệu trước"]')
            year_before.click()
            sleep(2)
    elif move_type == "after":
        for _ in range(times):
            year_after = driver.find_element(By.CSS_SELECTOR, 'img[alt="Xem dữ liệu tiếp"]')
            year_after.click()
            sleep(2)
    else:
        raise ValueError("move_type must be either 'before' or 'after'")
    sleep(2)

In [5]:
def get_table_data(ma,business_performance_results_table=False, balance_sheet=False, cash_flow_statement=False):
    result = {}
    df_result = pd.DataFrame(result)
    if business_performance_results_table == True:
        numbers = [f'0{i}' if i < 10 else str(i) for i in range(1, 61)]
        to_balance_sheet = driver.find_element(By.CSS_SELECTOR, 'a[id*="aNhom2"]')
        to_balance_sheet.click()
        sleep(2)

    elif balance_sheet == True:
        numbers = [i for i in range(100,441)]
        to_balance_sheet = driver.find_element(By.CSS_SELECTOR, 'a[id*="aNhom1"]')
        to_balance_sheet.click()
        sleep(2)

    elif cash_flow_statement == True:    
        numbers = [f'0{i}' if i < 10 else str(i) for i in range(1, 71)]
        
    else:
        return
    for number in numbers:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, f'tr[class*=r_item][id="{number}"] td')
            if elements == []:
                continue
        except:
            continue
        element_name = re.sub(r'^.*?\.', '', elements[0].text)
        element_name = re.sub(r'[^A-Za-zÀ-ỹ ]+', '', element_name).strip()
        elements_values = [element.text for element in elements[1:5]]
        result[f"{element_name}"] = elements_values
        
        element_years = driver.find_elements(By.CSS_SELECTOR, 'div[id="divTableHeader"] td')[1:5]
        result["years"] = [year.text for year in element_years]

        
        df_result = pd.DataFrame(result)
        df_result = df_result.replace('', None).apply(lambda col: col.str.replace(',', '').astype(float) if col.dtype == 'object' else col)
        df_result["ma"] = ma
        
    return df_result



# CODE EXECUTION BLOCK

In [6]:
global driver
driver = webdriver.Chrome()
driver.get("https://cafef.vn/du-lieu/bao-cao-tai-chinh/scd/incsta/2024/1/0/0/bao-cao-tai-chinh-.chn")

In [ ]:


# Danh sách mã cổ phiếu
codes = ['hpg', 'VIC', 'PHC', 'PIT', 'PJT', 'S4A', 'SAB', 'SAM', 'SAV', 'SBA', 'SBG', 'SBT', 'SBV', 'SC5',  'SCR', 'SCS']

df_business_performance, df_balance_sheet = pd.DataFrame(), pd.DataFrame()
for code in codes:
    driver.get(f"https://cafef.vn/du-lieu/bao-cao-tai-chinh/{code}/incsta/2024/0/0/0/0/bao-cao-tai-chinh-.chn")
    sleep(2)

    year_choosing()

    number_of_year_list = [4] * (NUMBER_OF_YEAR // 4) + ([NUMBER_OF_YEAR % 4] if NUMBER_OF_YEAR % 4 != 0 else [])
    for idx,num_year in enumerate(number_of_year_list):
        if idx != 0:
            year_monitoring("before", times=num_year)

        business_performance_data = get_table_data(ma=code, business_performance_results_table=True)
        df_business_performance = business_performance_data if df_business_performance.empty else pd.concat([df_business_performance, business_performance_data], axis=0).drop_duplicates(subset=['years', 'ma'], keep='last')

        balance_sheet_data = get_table_data(ma=code, balance_sheet=True)
        df_balance_sheet = balance_sheet_data if df_balance_sheet.empty else pd.concat([df_balance_sheet, balance_sheet_data], axis=0).drop_duplicates(subset=['years', 'ma'], keep='last')

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

df_business_performance['quy'] = 0
df_business_performance.to_csv(f'data/finance_data/business_performance_{current_time}.csv', index=False)

df_balance_sheet['quy'] = 0
df_balance_sheet.to_csv(f'data/finance_data/balance_sheet_{current_time}.csv', index=False)

display(df_business_performance)
display(df_balance_sheet)




print("Dữ liệu của tất cả các công ty đã được lưu vào sheet 'TongHop' trong file 'du_lieu_2018_2023.xlsx'")


,Doanh thu bán hàng và cung cấp dịch vụ,years,Các khoản giảm trừ doanh thu,Doanh thu thuần về bán hàng và cung cấp dịch vụ,Giá vốn hàng bán,Lợi nhuận gộp về bán hàng và cung cấp dịch vụ,Doanh thu hoạt động tài chính,Chi phí tài chính,Trong đó Chi phí lãi vay,Phần lãi lỗ trong công ty liên doanh liên kết,...,Chi phí quản lý doanh nghiệp,Lợi nhuận thuần từ hoạt động kinh doanh,Thu nhập khác,Chi phí khác,Lợi nhuận khác,Tổng lợi nhuận kế toán trước thuế,Chi phí thuế TNDN hiện hành,Chi phí thuế TNDN hoãn lại,Lợi nhuận sau thuế thu nhập doanh nghiệp,ma
0,1.508654e+14,2021.0,1.185570e+12,1.496798e+14,1.085714e+14,4.110841e+13,3.071441e+12,3.731542e+12,2.525823e+12,4.465303e+09,...,1.324262e+12,3.700844e+13,7.966661e+11,7.483318e+11,4.833427e+10,3.705678e+13,2.855306e+12,-3.194836e+11,3.452095e+13,hpg
1,1.427708e+14,2022.0,1.361536e+12,1.414093e+14,1.246458e+14,1.676343e+13,3.743651e+12,7.026723e+12,3.083638e+12,-1.072668e+09,...,1.019444e+12,9.794031e+12,8.720247e+11,7.431142e+11,1.289105e+11,9.922941e+12,1.001020e+12,4.774918e+11,8.444429e+12,hpg
2,1.203552e+14,2023.0,1.402204e+12,1.189530e+14,1.060152e+14,1.293784e+13,3.173086e+12,5.191512e+12,3.585078e+12,NaN,...,1.307290e+12,7.650763e+12,7.719735e+11,6.300075e+11,1.419660e+11,7.792729e+12,1.073552e+12,-8.121146e+10,6.800388e+12,hpg
3,1.405614e+14,2024.0,1.706275e+12,1.388551e+14,1.203576e+14,1.849755e+13,2.618917e+12,3.966636e+12,2.287361e+12,NaN,...,1.545516e+12,1.326701e+13,1.159770e+12,7.332730e+11,4.264967e+11,1.369350e+13,1.765806e+12,-9.232737e+10,1.202002e+13,hpg
0,4.685483e+13,2017.0,6.931341e+11,4.616169e+13,3.553612e+13,1.062557e+13,1.862155e+11,5.557566e+11,4.797076e+11,-1.596116e+08,...,4.090382e+11,9.252124e+12,4.327918e+11,3.965463e+11,3.624546e+10,9.288370e+12,1.317629e+12,-4.401563e+10,8.014757e+12,hpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1.036882e+12,2024.0,NaN,1.036882e+12,2.360628e+11,8.008193e+11,5.654197e+10,1.140912e+09,NaN,NaN,...,7.114068e+10,7.850797e+11,5.158712e+08,2.910699e+09,-2.394828e+09,7.826848e+11,8.985697e+10,NaN,6.928279e+11,SCS
0,5.883173e+11,2017.0,NaN,5.883173e+11,1.354427e+11,4.528747e+11,5.020425e+09,5.195811e+09,3.590520e+09,NaN,...,6.557189e+10,3.871274e+11,8.863295e+08,3.574968e+09,-2.688638e+09,3.844388e+11,3.997760e+10,NaN,3.444612e+11,SCS
1,6.753690e+11,2018.0,NaN,6.753690e+11,1.469889e+11,5.283802e+11,7.950282e+09,3.395337e+08,1.456335e+08,NaN,...,6.717649e+10,4.688144e+11,3.239010e+08,2.428037e+09,-2.104136e+09,4.667103e+11,5.099810e+10,NaN,4.157122e+11,SCS
2,7.479761e+11,2019.0,NaN,7.479761e+11,1.530648e+11,5.949113e+11,1.443502e+10,1.253093e+09,9.738857e+08,NaN,...,6.812196e+10,5.399713e+11,7.649689e+07,2.368615e+09,-2.292118e+09,5.376792e+11,3.549900e+10,NaN,5.021802e+11,SCS


,A TÀI SẢN NGẮN HẠN,years,Tiền và các khoản tương đương tiền,Tiền,Các khoản tương đương tiền,Các khoản đầu tư tài chính ngắn hạn,Chứng khoán kinh doanh,Dự phòng giảm giá chứng khoán kinh doanh,Đầu tư nắm giữ đến ngày đáo hạn,Các khoản phải thu ngắn hạn,...,Quỹ hỗ trợ sắp xếp doanh nghiệp,Quỹ khác thuộc vốn chủ sở hữu,Lợi nhuận sau thuế chưa phân phối,Nguồn vốn đầu tư XDCB,Lợi ích cổ đông không kiểm soát,Nguồn kinh phí và quỹ khác,Nguồn kinh phí,Nguồn kinh phí đã hình thành TSCĐ,TỔNG CỘNG NGUỒN VỐN,ma
0,9.415486e+13,2021.0,2.247138e+13,6.316300e+12,1.615508e+13,1.823615e+13,NaN,NaN,NaN,7.662681e+12,...,NaN,NaN,4.176343e+13,NaN,1.547887e+11,NaN,NaN,NaN,1.782364e+14,hpg
1,8.051471e+13,2022.0,8.324589e+12,3.458050e+12,4.866539e+12,2.626825e+13,NaN,NaN,NaN,9.892867e+12,...,NaN,NaN,3.383383e+13,NaN,1.055621e+11,NaN,NaN,NaN,1.703355e+14,hpg
2,8.271644e+13,2023.0,1.225200e+13,3.771595e+12,8.480406e+12,2.217730e+13,NaN,NaN,4.000000e+10,1.070214e+13,...,NaN,NaN,4.059303e+13,NaN,6.576985e+10,NaN,NaN,NaN,1.877826e+14,hpg
3,8.667428e+13,2024.0,6.887646e+12,2.919532e+12,3.968114e+12,1.897472e+13,NaN,NaN,1.365000e+11,7.647800e+12,...,NaN,NaN,4.959912e+13,NaN,2.909906e+11,NaN,NaN,NaN,2.244897e+14,hpg
0,3.306806e+13,2017.0,4.264642e+12,7.643970e+11,3.500245e+12,9.936707e+12,NaN,NaN,1.251738e+09,6.555419e+12,...,NaN,NaN,1.339703e+13,NaN,1.106138e+11,NaN,NaN,NaN,5.302218e+13,hpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1.388384e+12,2024.0,3.172886e+11,1.252886e+11,1.920000e+11,9.540000e+11,NaN,NaN,NaN,1.114481e+11,...,NaN,NaN,3.703869e+11,NaN,NaN,NaN,NaN,NaN,1.897525e+12,SCS
0,2.362453e+11,2017.0,4.731366e+10,4.731366e+10,NaN,1.240000e+11,NaN,NaN,NaN,6.330871e+10,...,NaN,NaN,3.198446e+11,NaN,NaN,NaN,NaN,NaN,9.896313e+11,SCS
1,2.133924e+11,2018.0,4.732084e+10,4.732084e+10,NaN,3.000000e+10,NaN,NaN,NaN,1.326911e+11,...,NaN,NaN,2.161457e+11,NaN,NaN,NaN,NaN,NaN,9.165794e+11,SCS
2,4.142367e+11,2019.0,6.609889e+10,6.609889e+10,NaN,1.800000e+11,NaN,NaN,NaN,1.265361e+11,...,NaN,NaN,3.849120e+11,NaN,NaN,NaN,NaN,NaN,1.073273e+12,SCS


Dữ liệu của tất cả các công ty đã được lưu vào sheet 'TongHop' trong file 'du_lieu_2018_2023.xlsx'
